In [52]:
# -*- coding: UTF-8 -*- 
import onnx
from onnx import helper, checker
from onnx import TensorProto
import re
import argparse
import json,copy
import numpy as np
from data_json import *
from onnx_split import *

In [53]:
origin_model = '/home/sky/Downloads/bvlcalexnet-9.onnx'
platform_file = './4platform.txt'


In [54]:
graph = load_onnx(origin_model)

Check input model Errors:  None


In [55]:
with open(platform_file, 'r') as f:         
        platform = f.readlines()
        #print (platform)
        for i in range(len(platform)):
            temp = platform[i].split(':')[0]
            platform[i] = temp.replace('\n','')
#print (platform)
platform_num = len(platform)

platform_dict={} 
######platform 对应id
for i in range(platform_num):
    platform_dict[platform[i]] = i
    
print ("platform dict: ", platform_dict)


platform dict:  {'nx01': 0, 'nx02': 1, 'nx03': 2, 'nx04': 3}


In [56]:
input_map = generate_node_dict(graph.input)  
output_map = generate_node_dict(graph.output)
initializer_map = generate_node_dict(graph.initializer)
value_map = generate_node_dict(graph.value_info)       
node_map = generate_node_dict(graph.node)


In [57]:
node_len = len(list(node_map))
mapping_dict ={}
nodes_number = 2

In [58]:
node_list = []
start_index = 0
end_index = 0

for i in range(nodes_number):  
    
    start_index = end_index
    end_index = min((int)(node_len/nodes_number)*(i+1) , node_len)
    #print (start_index, end_index)
    for j in range(start_index, end_index):
        node_list.append(graph.node[j].name)
    new_node_list = copy.copy(node_list)
    mapping_dict['nx0'+str(i+1)]=new_node_list
    node_list = []

In [51]:
print (mapping_dict)
save_json(mapping_dict,"./"+str(nodes_number)+"mapping.json")

{'nx01': ['conv1_1', 'conv1_2', 'norm1_1', 'pool1_1', 'conv2_1', 'conv2_2', 'norm2_1', 'pool2_1', 'conv3_1', 'conv3_2', 'conv4_1', 'conv4_2'], 'nx02': ['conv5_1', 'conv5_2', 'pool5_1', 'OC2_DUMMY_0', 'fc6_1', 'fc6_2', 'fc6_3', 'fc7_1', 'fc7_2', 'fc7_3', 'fc8_1', 'prob_1']}
./2mapping.json.json saved.
